In [17]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from nltk import word_tokenize
from collections import Counter
from string import punctuation
punct = punctuation+'©«»—…“”*№–'
import nltk
nltk.download('punkt')
import urllib.request
from bs4 import BeautifulSoup
import re
from tqdm import tqdm

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
def which_language (text):
  import re
  if bool(re.search('[\u0400-\u04FF]', text)) == True:
    return 'russian'
  else:
    return 'english'

In [0]:
teams_names_in_russian_and_english_df = pd.read_csv("teams names in russian and english.csv")
leagues_names_in_russian_and_english_df = pd.read_csv("leagues names in russian and english.csv")

In [20]:
df = pd.read_csv('result.tar.gz', sep='\t', error_bad_lines=False)

b'Skipping line 37039: expected 12 fields, saw 15\nSkipping line 45611: expected 12 fields, saw 15\n'
b'Skipping line 354339: expected 12 fields, saw 18\nSkipping line 360804: expected 12 fields, saw 102\nSkipping line 360819: expected 12 fields, saw 78\nSkipping line 379126: expected 12 fields, saw 14\nSkipping line 444446: expected 12 fields, saw 16\nSkipping line 449079: expected 12 fields, saw 16\nSkipping line 450250: expected 12 fields, saw 15\n'
b'Skipping line 481374: expected 12 fields, saw 18\nSkipping line 510619: expected 12 fields, saw 20\nSkipping line 512271: expected 12 fields, saw 14\nSkipping line 514317: expected 12 fields, saw 15\nSkipping line 519707: expected 12 fields, saw 40\n'
b'Skipping line 538495: expected 12 fields, saw 13\n'
b'Skipping line 765901: expected 12 fields, saw 56\nSkipping line 773310: expected 12 fields, saw 43\n'
b'Skipping line 805899: expected 12 fields, saw 13\nSkipping line 868492: expected 12 fields, saw 21\nSkipping line 871801: expecte

In [0]:
df = df.drop(["url", "video_url", "source_name", "author_name"], axis=1)

In [0]:
df['result.tsv'] = df['result.tsv'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True).replace(r'https\S+', '', regex=True)
df = df.replace('', np.NaN)
df = df.replace('\xa0', np.NaN)
df = df.dropna()
df =  df[df['result.tsv'].str.contains('[A-Za-z]')]
df = df[df['result.tsv'].str.split().str.len().gt(1)]

In [0]:
df = df.dropna()

In [0]:
df = df.drop_duplicates()

In [0]:
df = df.reset_index(drop=True)

In [26]:
df.head(5)

,result.tsv,odd,tip_name,bookmaker_name,match_date_time,league_name_in_russian,team_of_season_1_name_in_russian,team_of_season_2_name_in_russian
0,Я предполагаю ничью в этом матче с забитыми мя...,5,Арсенал забьет первым и не выиграет,1xСтавка,2019-05-09 19:00:00,Лига Европы УЕФА,Валенсия,Арсенал
1,Беларусь – Эстония: прогноз и ставка на матч 1...,1.9,"Беларусь индивидуальный тотал больше 1,5",Пари-Матч,2019-10-10 16:00:00,Чемпионат Европы. Квалификация,Беларусь,Эстония
2,Украинская команда имеет отличный шанс выйти в...,1.55,Заря забьет 1 или 2 гола: да,1xСтавка,2019-08-15 17:00:00,Лига Европы УЕФА,Заря,ЦСКА София
3,«Сент-Миррен» в родных стенах встретится с «Аб...,2.55,«Абердин» фора (-1),BETCITY,2019-08-11 14:00:00,Премьершип,Сент-Миррен,Абердин
4,На стадионе «Айброкс» состоится встреча между ...,1.97,"«Абердин» фора (+1,5)",Марафон,2019-09-28 14:00:00,Премьершип,Глазго Рейнджерс,Абердин


In [0]:
lang=[]
for i,text in enumerate(df['result.tsv'].values): 
  lang.append(which_language(str(text)))

In [0]:
df['Language'] = lang

In [0]:
grouped=df.groupby('Language')

In [0]:
for name, group in grouped: 
  if name == 'english': 
    english = group.drop('Language',axis=1) 
  else: 
    russian = group.drop('Language',axis=1)

In [0]:
english = pd.merge(english, teams_names_in_russian_and_english_df,  how='left', left_on=['team_of_season_1_name_in_russian'], right_on = ['team name in Russian'])
english.rename({'English': 'team_of_season_1_name_in_english'}, axis=1, inplace=True)

english = pd.merge(english, teams_names_in_russian_and_english_df,  how='left', left_on=['team_of_season_2_name_in_russian'], right_on = ['team name in Russian'])
english.rename({'English': 'team_of_season_2_name_in_english'}, axis=1, inplace=True)

english = pd.merge(english, leagues_names_in_russian_and_english_df,  how='left', left_on=['league_name_in_russian'], right_on = ['leagues name in Russian'])
english.rename({'unique_leagues_in_english': 'league_name_in_english'}, axis=1, inplace=True)


In [0]:
english.drop(['team name in Russian_x' , 'team name in Russian_y' , 'leagues name in Russian' ] , axis=1 , inplace=True)

In [0]:
# Here we are just re-naming the columns so one check that what's in it and interpret the information easily 
english.rename({'team name in English_x': 'team_of_season_1_name_in_english'}, axis=1, inplace=True)
english.rename({'team name in English_y': 'team_of_season_2_name_in_english'}, axis=1, inplace=True)
english.rename({'leagues names in English': 'league_name_in_english'}, axis=1, inplace=True)

In [0]:
# Here we are just setting the correct data type that english names should be string
english['team_of_season_1_name_in_english']=english['team_of_season_1_name_in_english'].astype(str)
english['team_of_season_2_name_in_english']=english['team_of_season_2_name_in_english'].astype(str)
english['league_name_in_english']=english['league_name_in_english'].astype(str)

In [35]:
english.head(2)

,result.tsv,odd,tip_name,bookmaker_name,match_date_time,league_name_in_russian,team_of_season_1_name_in_russian,team_of_season_2_name_in_russian,team_of_season_1_name_in_english,team_of_season_2_name_in_english,league_name_in_english
0,Middlesbrough have woken to good times in thei...,7,0-0,bet365,2019-04-19 14:00:00,Чемпионшип,Мидлсбро,Сток Сити,Middlesbrough,Stoke City,Championship
1,Wolves prefer to attack with pace on the count...,7.5,0-0,bet365,2019-04-20 14:00:00,Премьер-лига,Вулверхэмптон,Брайтон,Wolverhampton,Brighton,Premier League



**Home page link with football clubs by country**


In [0]:
r = urllib.request.urlopen('https://en.wikipedia.org/wiki/Category:Association_football_clubs_by_country')

In [0]:
result = r.read()

In [0]:
content = result.decode('utf-8')

In [0]:
soup = BeautifulSoup(content, 'lxml') 

In [0]:
links = []


We parse it and get links to individual countries.


In [0]:
for div in soup.find_all('div', attrs={'class':'CategoryTreeItem'}):
    link = div.find('a').get('href')
    links.append(link)


In [42]:
len(links)

200

**Per country**


In [43]:
teams_dict = {}
empty_team_lists = [] #list for country teams with a different page structure
empty_team_lists_links = []

for link in tqdm(links):
    page = urllib.request.urlopen('https://en.wikipedia.org/' + link) #follow every link
    result = page.read()
    content = result.decode('utf-8')
    soup = BeautifulSoup(content, 'lxml')
    for h1 in soup.find_all('h1', attrs={'class':'firstHeading'}): #find the right item in html
        name = h1.get_text() #we take the name, e.g.: Football Clubs of England
        if name.startswith('Category:'):
            norm_name = name.replace('Category:', '')
            
    teams_list = []  
    for div in soup.find_all('div', attrs={'class':'mw-category-group'}): #we find all the teams in this country
        team_links = div.find_all('a')
        for team_link in team_links:
            team = team_link.get('title')
            #print(team)
            if team.startswith('Category:FC'):
                norm_team = team.replace('Category:FC', '')
                norm_team = re.sub(r'[«»]','', norm_team)
                norm_team = norm_team.strip(' ')
                norm_team = re.sub(r'\(football club.+\)','', norm_team)
                norm_team = re.sub(r'\(football club\)','', norm_team)
                norm_team = re.sub(r'\(sports community\)','', norm_team)
                norm_team = re.sub(r'\(women football club\)','', norm_team)
                norm_team = re.sub(r'\(women football club.+\)','', norm_team)
                norm_team = norm_team.strip(' ')
                #norm_team = norm_team.strip(punctuation)
            
            elif team.startswith('Category:') or team.startswith('List'):
                    continue
                
            elif team.endswith('(football club)'):
                norm_team = team.replace('(football club)', '')
                norm_team = re.sub(r'[«»]','', norm_team)
                norm_team = norm_team.strip(' ')
                norm_team = re.sub(r'\(football club.+\)','', norm_team)
                norm_team = re.sub(r'\(football club\)','', norm_team)
                norm_team = re.sub(r'\(sports community\)','', norm_team)
                norm_team = re.sub(r'\(women football club\)','', norm_team)
                norm_team = re.sub(r'\(women football club.+\)','', norm_team)
                norm_team = norm_team.strip(' ')
                #norm_team = norm_team.strip(punctuation)
            else:
                norm_team = team
                norm_team = re.sub(r'[«»]','', norm_team)
                norm_team = norm_team.strip(' ')
                norm_team = re.sub(r'\(football club.+\)','', norm_team)
                norm_team = re.sub(r'\(football club\)','', norm_team)
                norm_team = re.sub(r'\(sports community\)','', norm_team)
                norm_team = re.sub(r'\(women football club\)','', norm_team)
                norm_team = re.sub(r'\(women football club.+\)','', norm_team)
                norm_team = norm_team.strip(' ')
                #norm_team = norm_team.strip(punctuation)
            
            teams_list.append(norm_team)
    teams_list1 = teams_list
    
    teams_list2 = []
    second_page_div = soup.find('div', attrs={'id':'mw-pages'})
    if second_page_div:
        a = second_page_div.find('a')
        if a.get_text() == 'Next page':
            second_page = a.get('href')
            link = 'https://en.wikipedia.org'+second_page
            page2 = urllib.request.urlopen(link)
            result2 = page2.read()
            content2 = result2.decode('utf-8')
            soup2 = BeautifulSoup(content2, 'lxml')
            h2 = soup2.find('div', attrs={'id':'mw-pages'})
            for a in h2.find_all('a'):
                team2 = a.get('title')
                #print(team2)
                if team2.startswith('Category:FC'):
                    norm_team2 = team2.replace('Category:FC', '')
                    norm_team2 = re.sub(r'[«»]','', norm_team2)
                    norm_team2 = norm_team2.strip(' ')
                    norm_team2 = re.sub(r'\(football club.+\)','', norm_team2)
                    norm_team2 = re.sub(r'\(football club\)','', norm_team2)
                    norm_team2 = re.sub(r'\(sports community\)','', norm_team2)
                    norm_team2 = re.sub(r'\(women football club\)','', norm_team2)
                    norm_team2 = re.sub(r'\(women football club.+\)','', norm_team2)
                    norm_team2 = norm_team2.strip(' ')
                    #norm_team2 = norm_team2.strip(punctuation)
                elif team2.startswith('Category:') or team2.startswith('Список'):
                    continue    
                elif team.endswith('(football club)'):
                    norm_team2 = team2.replace('(football club)', '')
                    norm_team2 = re.sub(r'[«»]','', norm_team2)
                    norm_team2 = norm_team2.strip(' ')
                    norm_team2 = re.sub(r'\(football club.+\)','', norm_team2)
                    norm_team2 = re.sub(r'\(football club\)','', norm_team2)
                    norm_team2 = re.sub(r'\(sports community\)','', norm_team2)
                    norm_team2 = re.sub(r'\(women football club\)','', norm_team2)
                    norm_team2 = re.sub(r'\(women football club.+\)','', norm_team2)
                    norm_team2 = norm_team2.strip(' ')
                    #norm_team2 = norm_team2.strip(punctuation)
                else:
                    norm_team2 = team2
                    norm_team2 = re.sub(r'[«»]','', norm_team2)
                    norm_team2 = norm_team2.strip(' ')
                    norm_team2 = re.sub(r'\(football club.+\)','', norm_team2)
                    norm_team2 = re.sub(r'\(football club\)','', norm_team2)
                    norm_team2 = re.sub(r'\(sports community\)','', norm_team2)
                    norm_team2 = re.sub(r'\(women football club\)','', norm_team2)
                    norm_team2 = re.sub(r'\(women football club.+\)','', norm_team2)
                    norm_team2 = norm_team2.strip(' ')
                    #norm_team2 = norm_team2.strip(punctuation)
                #print(norm_team2)
                
                teams_list2.append(norm_team2)
            #print(teams_list2)
    
    if teams_list2:
        full_teams_list = set(teams_list1 + teams_list2)
    else:
        full_teams_list = set(teams_list1)
                

    if full_teams_list: #if the command list is not empty, we add it to the dictionary.
        teams_dict[norm_name] = full_teams_list
    else:
        empty_team_lists.append(norm_name) #If the command list is empty, we add it to the list with the same countries.
        empty_team_lists_links.append(link)
    

100%|██████████| 200/200 [01:00<00:00,  3.30it/s]


**Adding remaining clubs to the dictionary**

In [44]:
bad_links = []

for link in tqdm(empty_team_lists_links):
    #print('https://en.wikipedia.org' + link)
    page = urllib.request.urlopen('https://en.wikipedia.org' + link) 
    result = page.read()
    content = result.decode('utf-8')
    soup = BeautifulSoup(content, 'lxml')
    for h1 in soup.find_all('h1', attrs={'class':'firstHeading'}): #find the right item in html 
        name = h1.get_text() #we take the name, e.g.: Football Clubs of England.
        if name.startswith('Category:'):
            norm_name = name.replace('Category:', '')
    
    teams_list = []    
    for div in soup.find_all('div', attrs={'class':'mw-category-generated'}): #we find all the teams in this country
        #print(div)
        team_links = div.find_all('a')
        #print(team_links)
        for team_link in team_links:
            team = team_link.get('title')
            if team.startswith('Category:FC'):
                norm_team = team.replace('Category:FC', '') 
                norm_team = re.sub(r'[«»]','', norm_team)
                norm_team = norm_team.strip(' ')
                norm_team = re.sub(r'\(football club.+\)','', norm_team)
                norm_team = re.sub(r'\(football club\)','', norm_team)
                norm_team = re.sub(r'\(sports community\)','', norm_team)
                norm_team = re.sub(r'\(women football club\)','', norm_team)
                norm_team = re.sub(r'\(women football club.+\)','', norm_team)
                norm_team = norm_team.strip(' ')
                
            elif team.endswith('(football club)'):
                norm_team = team.replace('(football club)', '')
                norm_team = re.sub(r'[«»]','', norm_team)
                norm_team = norm_team.strip(' ')
                norm_team = re.sub(r'\(football club.+\)','', norm_team)
                norm_team = re.sub(r'\(football club\)','', norm_team)
                norm_team = re.sub(r'\(sports community\)','', norm_team)
                norm_team = re.sub(r'\(women football club\)','', norm_team)
                norm_team = re.sub(r'\(women football club.+\)','', norm_team)
                norm_team = norm_team.strip(' ')
                
            elif team.startswith('Category:'):
                bad_links.append(team_link.get('href'))
                #norm_team = norm_team.strip(' ')
                
            else:
                norm_team = team
                norm_team = re.sub(r'[«»]','', norm_team)
                norm_team = norm_team.strip(' ')
                norm_team = re.sub(r'\(football club.+\)','', norm_team)
                norm_team = re.sub(r'\(football club\)','', norm_team)
                norm_team = re.sub(r'\(sports community\)','', norm_team)
                norm_team = re.sub(r'\(women football club\)','', norm_team)
                norm_team = re.sub(r'\(women football club.+\)','', norm_team)
                norm_team = norm_team.strip(' ')
                    
            teams_list.append(norm_team)
    teams_list = set(teams_list)
    teams_dict[norm_name] = teams_list

100%|██████████| 18/18 [00:02<00:00,  6.04it/s]



**Remaining bad references**

In [45]:
for link in bad_links[:3]:
    print('https://en.wikipedia.org' + link)

https://en.wikipedia.org/wiki/Category:Lists_of_football_clubs_in_China
https://en.wikipedia.org/wiki/Category:Lists_of_football_clubs_in_Finland
https://en.wikipedia.org/wiki/Category:Lists_of_football_clubs_in_Germany



**Parsing clubs of a particular country/city by link**


In [0]:
link = 'https://en.wikipedia.org/wiki/Category:Football_clubs_in_Kiev'
page = urllib.request.urlopen(link) 
result = page.read()
content = result.decode('utf-8')
soup = BeautifulSoup(content, 'lxml')
for h1 in soup.find_all('h1', attrs={'class':'firstHeading'}): #find the right item in html 
        name = h1.get_text() #we take the name, e.g.: Football Clubs of England.
        if name.startswith('Category:'):
            norm_name = name.replace('Category:', '')
            
teams_list = []    
for div in soup.find_all('div', attrs={'class':'mw-category-generated'}): #we find all the teams in this country
    #print(div)
    team_links = div.find_all('a')
    #print(team_links)
    for team_link in team_links:
        team = team_link.get('title')
        if team.startswith('Category:FC'):
            norm_team = team.replace('Category:FC', '') 
            norm_team = re.sub(r'[«»]','', norm_team)
            norm_team = norm_team.strip(' ')
            norm_team = re.sub(r'\(football club.+\)','', norm_team)
            norm_team = re.sub(r'\(football club\)','', norm_team)
            norm_team = re.sub(r'\(sports community\)','', norm_team)
            norm_team = re.sub(r'\(women football club\)','', norm_team)
            norm_team = re.sub(r'\(women football club.+\)','', norm_team)
            norm_team = norm_team.strip(' ')
                
        elif team.endswith('(football club)'):
            norm_team = team.replace('(football club)', '')
            norm_team = re.sub(r'[«»]','', norm_team)
            norm_team = norm_team.strip(' ')
            norm_team = re.sub(r'\(football club.+\)','', norm_team)
            norm_team = re.sub(r'\(football club\)','', norm_team)
            norm_team = re.sub(r'\(sports community\)','', norm_team)
            norm_team = re.sub(r'\(women football club\)','', norm_team)
            norm_team = re.sub(r'\(women football club.+\)','', norm_team)
            norm_team = norm_team.strip(' ')
                
        elif team.startswith('Category:'):
            bad_links.append(team_link.get('href'))
            #norm_team = norm_team.strip(' ')
                
        else:
            norm_team = team
            norm_team = re.sub(r'[«»]','', norm_team)
            norm_team = norm_team.strip(' ')
            norm_team = re.sub(r'\(football club.+\)','', norm_team)
            norm_team = re.sub(r'\(football club\)','', norm_team)
            norm_team = re.sub(r'\(sports community\)','', norm_team)
            norm_team = re.sub(r'\(женский футбольный клуб\)','', norm_team)
            norm_team = re.sub(r'\(женский футбольный клуб.+\)','', norm_team)
            norm_team = norm_team.strip(' ')
                    
        teams_list.append(norm_team)
teams_list = set(teams_list)
teams_dict[norm_name] = teams_list

In [0]:
list_teams = [*teams_dict.values()]

In [0]:
count_teams = []
for team_set in list_teams:
    for team in team_set:
        count_teams.append(team)

In [0]:
uniq_teams = set(count_teams)


Match on the list of teams

In [0]:
teams1_match = np.zeros(english.shape[0])
teams2_match = np.zeros(english.shape[0])

In [0]:
for i, text in enumerate(english['result.tsv'].values):
    txt_tokens = []
    txt_tokens.append(word_tokenize(text))
    for txt in txt_tokens:
        count_tokens = []
        for token in txt:
            token = token.strip(punct)
            if token in uniq_teams:
                count_tokens.append(token)
        c = Counter(count_tokens)
        
        try: 
            max_w1 = sorted(c, key=c.get)[-1]
        except:
            continue
        
        try:
            max_w2 = sorted(c, key=c.get)[-2]

        except:
            continue
        
        if english['team_of_season_1_name_in_english'][i] == max_w1 or english['team_of_season_1_name_in_english'][i] == max_w2:
            teams1_match[i] = 1

        
        if english['team_of_season_2_name_in_english'][i] == max_w1 or english['team_of_season_2_name_in_english'][i] == max_w2:
            teams2_match[i] = 1


In [0]:
english['list_direct_match_for_teams1'] = teams1_match
english['list_direct_match_for_teams2'] = teams2_match

In [53]:
success1 = english[english['list_direct_match_for_teams1'] == 1].shape[0]
success2 = english[english['list_direct_match_for_teams2'] == 1].shape[0]

all_vals = english['list_direct_match_for_teams1'].shape[0]

result_team1 = success1 / all_vals
print(result_team1)

result_team2 = success2 / all_vals
print(result_team2)

0.0015235534644037416
0.0009522209152523386


In [0]:
english.to_csv('english_custom_lists_and_directmatch.csv', index = False)